In [1]:
# if (bm > 0.72 AND mom12m > 0.78) OR mve > 1.15:
#     signal = BUY

# SELL RULE:
# if (bm < -0.68 AND mom12m < -0.68) OR (mve < -1.6 AND mom12m < 0.1):
#     signal = SELL

#CAUSAL MASKING:
# BUY RULE:
# if (bm > 0.85 AND mom12m > -0.4 AND mve > -0.75):
#     signal = BUY

# SELL RULE:
# if (bm < -0.75 AND mom12m < -0.55 AND mve < 0.0):
#     signal = SELL

In [22]:
import pandas as pd
import numpy as np

green_cleaned = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
green_cleaned['ret_fwd_1'] = (green_cleaned.groupby('permno')['ret_excess'].shift(-1) )

In [23]:
variables = ['datadate', 'permno', 'ticker', 'comnam', 'bm', 'mom12m', 'mve', 'ret_fwd_1']

In [26]:
start_date = '2019-12-31'
end_date = '2024-11-30'

target_date = '2019-12-31'

# 3. Create the filtered DataFrame for your analysis
# analysis_df = green_cleaned[
#     (green_cleaned['datadate'] >= start_date) & 
#     (green_cleaned['datadate'] <= end_date)
# ][variables].copy()

analysis_df = green_cleaned[
    (green_cleaned['datadate'] == target_date)
][variables].copy()

conditions = [
    # BUY: bm > 0.75 AND mom12m > 0.35 AND mve > -0.5
    ( (analysis_df['bm'] > 0.85) & (analysis_df['mom12m'] > -0.4) ) & ((analysis_df['mve'] > -0.75) ),
    
    # SELL: bm < -0.5 AND mom12m < -0.75
    ( (analysis_df['bm'] < -0.75) & (analysis_df['mom12m'] < -0.55) ) & ( (analysis_df['mve'] < 0) )
]

# 3. Define choices
choices = ['BUY', 'SELL']

# 4. Create the 'signal' column
analysis_df['signal'] = np.select(conditions, choices, default='HOLD')

buys = analysis_df[analysis_df['signal'] == 'BUY'][['datadate', 'permno', 'ticker', 'comnam', 'ret_fwd_1']]
sells = analysis_df[analysis_df['signal'] == 'SELL'][['datadate', 'permno', 'ticker', 'comnam', 'ret_fwd_1']]


print("\n--- BUY ---")
with pd.option_context('display.max_rows', None):
    print(buys)


print("\n--- SELL ---")
print(sells)

buys.to_csv('buys.csv', index=False)
sells.to_csv('sells.csv', index=False)


--- BUY ---
          datadate  permno ticker                            comnam  ret_fwd_1
9657    2019-12-31   11404     ED           CONSOLIDATED EDISON INC   0.037718
17520   2019-12-31   12369     GM                 GENERAL MOTORS CO  -0.089005
20084   2019-12-31   12558    KMI                 KINDER MORGAN INC  -0.003662
32297   2019-12-31   14776    SYF               SYNCHRONY FINANCIAL  -0.101272
37388   2019-12-31   15707    HPE     HEWLETT PACKARD ENTERPRISE CO  -0.122990
84114   2019-12-31   24053   EVRG                        EVERGY INC   0.107319
101057  2019-12-31   26710      L                        LOEWS CORP  -0.021113
106582  2019-12-31   27959    DUK              DUKE ENERGY CORP NEW   0.069087
109624  2019-12-31   28484    HES                         HESS CORP  -0.154570
115387  2019-12-31   34746   FITB               FIFTH THIRD BANCORP  -0.075796
148474  2019-12-31   47896    JPM               JPMORGAN CHASE & CO  -0.045346
163026  2019-12-31   52708    LEN      